# Proyecto con OpenCV

# Detección de defectos en una pieza de automoción mediante opencv

### Explicación del problema

1) Descripción general del proyecto: Este proyecto tiene como objetivo desarrollar un sistema basado en visión por computadora capaz de inspeccionar e identificar de forma autónoma defectos o imperfecciones en piezas metálicas fundidas. 
El sistema utilizarà técnicas de visión artificial convencional con la libreria de opencv, con este tipo de sistema no buscamos la robustez pero si comprovar la eficacia de un sistema tradicional, a medida que vayamos realizando el proyecto, se sacará conclusiones si con solo opencv se puede llegar a una estabilidad del sistema.

In [1]:

# Primero visualizaremos imágenes ok y ko para ver la diferencia entre ellas, y exponer cual puede ser la mejor técnica para su detección.
# en la siguiente url encontraremos el proyecto madre junto con el dataset: https://datasetsearch.research.google.com/search?src=0&query=defects%20in%20metal%20piece&docid=L2cvMTF2eDVoamZibA%3D%3D

#importamos la librerias que serán necesarias para nuestro desarrollo

import cv2
import numpy as np
import matplotlib.pyplot as plt




In [22]:
#visualizamos una imagen ok y una ko

img_ok = cv2.imread(r'C:\Users\joano\Desktop\vision artificial juan olivan\mi codigo\proyecto_final\casting_data\ok\cast_ok_0_2474.jpeg')#Cargamos una imagen ok
img_defect = cv2.imread(r'C:\Users\joano\Desktop\vision artificial juan olivan\mi codigo\proyecto_final\casting_data\defect\cast_def_0_5248.jpeg')#Carga de imagen con defecto

#mostramos las imágenes

# cv2.imshow('OK',img_ok)
# cv2.imshow('Defect',img_defect)

#añadimos este comando para que no se cierre las ventanas hasta que no se presione cualquier botón
cv2.waitKey()
cv2.destroyAllWindows()

Como podemos observar en las anteriores imágenes la diferencia entre ok y el ko, es la rotura que se encuentra en la imagen ko
podemos averiguar que todos los defectos tienen la misma característica, son negros en un fondo plateado, por ende vamos a plantear la siguientes soluciones
teniendo en cuenta que sólo vamos a utilizar la potencia de opencv, sin realizar 'ingenieria de características'.

Posibles soluciones:
 1. Utilizar la técnica de contornos: en ella binarizaremos la imagen, teniendo en cuenta un umbral automático, buscaremos los contornos y los filtraremos mediante un rango
 2. Utilitzar la técnica de búscar el patrón: en este caso tambien binarizaremos la imagen, teniendo en cuenta un umbral autómatico, y buscaremos en la imagen un patrón con las mismas características 
esta técnica no es aconsejable utilizarla, ya que el defecto puede variar en tamaño y en forma, aunque veremos a ver como se comporta.

### 1. Técnica de contornos

In [23]:
#probaremos de utilizar la técnica con una sola imagen, cuando hayamos encontrado la solución lo prepraremos para que sea automático
#tendremos en cuenta la imagenes cargadas anteriormente y seguiremos los pasos siguiente.

#Pasar a escala de grises las imágenes 
gray_ok=cv2.cvtColor(img_ok,cv2.COLOR_BGR2GRAY)
gray_defect=cv2.cvtColor(img_defect,cv2.COLOR_BGR2GRAY)

#binarizamos la imagenes
_,th=cv2.threshold(gray_ok,120,255,cv2.THRESH_BINARY)#+cv2.THRESH_OTSU)#binarizamos la imagen de escala de grises
_,th2=cv2.threshold(gray_defect,120,255,cv2.THRESH_BINARY)#+cv2.THRESH_OTSU)#binarizamos la imagen de escala de grises

#mostramos binarizadas
# cv2.imshow('OK',th)
# cv2.imshow('Defect',th2)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [24]:
#aplicaremos un filtro de morofologia closing para potenciar el negro sobre lo blanco del disco, para ello necesitamos crear un kernel

kernel=np.ones((5,5),np.uint8) #creamos un kernel de matriz de 1's 5X5

closingImg=cv2.morphologyEx(th,cv2.MORPH_CLOSE,kernel)#aplicamos el filtro de closing a la imagen binarizada con un kernel (5,5)
closingImg2=cv2.morphologyEx(th2,cv2.MORPH_CLOSE,kernel)#aplicamos el filtro de closing a la imagen binarizada con un kernel (5,5)

#mostramos los resultados con el filtro closing
# cv2.imshow('OK',closingImg)
# cv2.imshow('Defect',closingImg2)
# cv2.waitKey()
# cv2.destroyAllWindows()

In [25]:
#aplicaremos el metodo de encontrar los contornos en la imagen 
contours,_=cv2.findContours(closingImg,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)#encontramos los contornos exteriores y usamos el método aprox simple para ello
contours2,_=cv2.findContours(closingImg2,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)#encontramos los contornos exteriores y usamos el método aprox simple para ello


In [26]:
#Recorremos la lista creada por los contornos de la imagen y lo filtraremos por area, iremos barajando cual es el area que queremos abarcar

contornosLista=[] #lista vacia para rellenar con las posiciones de cada contorno

for index in range(len(contours)): #creamos un bucle for para recorrer cada uno de los contornos encontrados
    area=cv2.contourArea(contours[index]) #deifinmos el area de cada contorno para posteriormente poder filtrarla según el tamaña que desemos
    if area<200: #escogemos el tamaño
        cnt=contours[index] #se identifica el contorno
        contornosLista.append(cnt)#se añade a la lista vacia
print(len(contornosLista))
#replicamos para la segunda imagen
contornosLista2=[]

for index in range(len(contours2)):
    area=cv2.contourArea(contours2[index])
    if area<200:
        cnt=contours2[index]
        contornosLista2.append(cnt)
print(len(contornosLista2))

0
4


In [27]:
if len(contornosLista)==0:
    cv2.putText(img_ok,"No hay defecto",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),1,cv2.LINE_AA)  
    print('pieza ok')
else:
    # cv2.putText(img_ok,"hay defecto",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
    cv2.drawContours(img_ok,contornosLista,-1,(0,0,255),1)#dibujamos los contornos en la imagen copia y los ponemos en color verde

if len(contornosLista2)==0:   
    cv2.putText(img_defect,"no hay defecto",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),1,cv2.LINE_AA)
else:
    cv2.drawContours(img_defect,contornosLista2,-1,(0,0,255),1)#dibujamos los contornos en la imagen copia y los ponemos en color verde
    cv2.putText(img_defect,"hay defecto",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)

cv2.namedWindow("imagen1",cv2.WINDOW_NORMAL)
cv2.namedWindow("imagen2",cv2.WINDOW_NORMAL)

cv2.imshow("imagen1",img_ok)
cv2.imshow("imagen2",img_defect)

cv2.waitKey()
cv2.destroyAllWindows()

pieza ok


Como hemos podido observar en las imagenes ok y ko mostradas, hemos conseguido mediante la detección de bordes encontrar el posible defecto en la pieza, ahora pasaremos a observar como se comporta esta posible solución con imágenes buenas e imagenes ko.

De esta manera planteamos el sistema de manera automática.

In [3]:
import os

# Ruta al directorio donde se encuentran las imágenes del dataset
directorio_ok = "C:/Users/joano/Desktop/vision artificial juan olivan/mi codigo/proyecto_final/casting_data/ok/"
directorio_defect = "C:/Users/joano/Desktop/vision artificial juan olivan/mi codigo/proyecto_final/casting_data/defect/"
directorio_todas = "C:/Users/joano/Desktop/vision artificial juan olivan/mi codigo/proyecto_final/casting_data/todas/"
# Lista de archivos en el directorio
archivos = os.listdir(directorio_todas)

# #imagenes decision
# pulgar_ok=cv2.imread(r'C:\Users\joano\Desktop\vision artificial juan olivan\mi codigo\proyecto_final\casting_cat\decision\pulgar_ok.png')
# pulgar_nok=cv2.imread(r'C:\Users\joano\Desktop\vision artificial juan olivan\mi codigo\proyecto_final\casting_cat\decision\pulgar_nok.png')
# Iteramos sobre cada archivo en el directorio
for archivo in archivos:
    # Comprobamos si el archivo es una imagen (extensión .jpg o .png)
    if archivo.endswith(".jpeg"):
        # Construimos la ruta completa de la imagen
        imagen_path = os.path.join(directorio_todas, archivo)

        # Cargamos la imagen
        imagen = cv2.imread(imagen_path)

      
        # Convertimos la imagen a escala de grises
        imagen_gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)



        # Aplicamos un umbral para resaltar los defectos
        _,th=cv2.threshold(imagen_gris,120,255,cv2.THRESH_BINARY)#+cv2.THRESH_OTSU)#binarizamos la imagen de escala de grises

        kernel=np.ones((5,5),np.uint8) #creamos un kernel de matriz de 1's 5X5

        closingImg=cv2.morphologyEx(th,cv2.MORPH_CLOSE,kernel)
        
        # Encontramos los contornos de los defectos
        contours,_=cv2.findContours(closingImg,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)#encontramos los contornos exteriores y usamos el método aprox simple para ello

        contornosLista=[] #lista vacia para rellenar con las posiciones de cada contorno

        for index in range(len(contours)): #creamos un bucle for para recorrer cada uno de los contornos encontrados
            area=cv2.contourArea(contours[index]) #deifinmos el area de cada contorno para posteriormente poder filtrarla según el tamaña que desemos
            if 50<area<200: #escogemos el tamaño
                cnt=contours[index] #se identifica el contorno
                contornosLista.append(cnt)#se añade a la lista vacia
        
        imagen_defectos = imagen.copy()

        
        
        if len(contornosLista)==0:   
            cv2.putText(imagen,"OK",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),1,cv2.LINE_AA)
 
        else:
            cv2.drawContours(imagen_defectos,contornosLista,-1,(0,0,255),1)#dibujamos los contornos en la imagen copia y los ponemos en color verde
            cv2.putText(imagen_defectos,"NO OK",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),1,cv2.LINE_AA)
            
        # Mostramos la imagen original y la imagen con los defectos detectados
        cv2.imshow("Imagen original", imagen)
        cv2.imshow("Defectos detectados", imagen_defectos)
        
        # Esperamos una tecla para pasar a la siguiente imagen
        tecla = cv2.waitKey(0)
        
        # Si se presiona la tecla 'q' (quit), salimos del bucle
        if tecla == ord('q'):
            break

cv2.destroyAllWindows()


#### 2. Técnica de patrones

In [2]:
#Como hemos echo con el ejemplo anterior utilizaremos la técnica de patrónes para ver como se comporta en una imagen ko, y una vez se haga el exploratorio pasaremos a inspeccionar cada imagen.

def umbral(valor):
    img2=imgRgb.copy()
    w,h=template.shape[::-1]
    res=cv2.matchTemplate(imgGray,template,cv2.TM_CCOEFF_NORMED)
    umbralX=valor/100
    rectangulo=np.where(res>=umbralX)
    for puntos in zip(*rectangulo[::-1]):
        # print(puntos)
        cv2.rectangle(img2,puntos,(puntos[0]+w,puntos[1]+h),(0,255,0),2)
    cv2.imshow('match',img2)




cv2.namedWindow('match',cv2.WINDOW_NORMAL)

imgRgb=cv2.imread(r'C:\Users\joano\Desktop\vision artificial juan olivan\mi codigo\proyecto_final\casting_data\defect\cast_def_0_3028.jpeg')

roi=cv2.selectROI('match',imgRgb,False) #creamos un template a través de la selección en la pantalla 
x=roi[0]
y=roi[1]
w=roi[2]
h=roi[3]

template=imgRgb[y:y+h,x:x+w]#creamos la imagen recortada seleccionada en la imagen
imagen_grabada=cv2.imwrite(r'C:\Users\joano\Desktop\vision artificial juan olivan\mi codigo\proyecto_final\casting_data\template\template.jpeg',template)
print("Imagen grabada")

imgGray=cv2.cvtColor(imgRgb,cv2.COLOR_BGR2GRAY)
template=cv2.cvtColor(template,cv2.COLOR_BGR2GRAY)



# template=cv2.imread(r'C:\Users\joano\Desktop\vision artificial juan olivan\imagenes\template.jpg',0)#Carga de imagen original
cv2.imshow('match',imgRgb)

cv2.createTrackbar('Umbral','match',0,100,umbral)


cv2.waitKey()
cv2.destroyAllWindows()



Imagen grabada


Después de observar el exploratorio creando un template de un defecto en la imagen podemos decidir que puede funcionar, aunque con un valor de umbral de 80.

In [4]:
import os

# Ruta al directorio donde se encuentran las imágenes del dataset
directorio_ok = "C:/Users/joano/Desktop/vision artificial juan olivan/mi codigo/proyecto_final/casting_data/ok/"
directorio_defect = "C:/Users/joano/Desktop/vision artificial juan olivan/mi codigo/proyecto_final/casting_data/defect/"
directorio_todas = "C:/Users/joano/Desktop/vision artificial juan olivan/mi codigo/proyecto_final/casting_data/todas/"
template_path="C:/Users/joano/Desktop/vision artificial juan olivan/mi codigo/proyecto_final/casting_data/template/template.jpeg" #hemos obtenido esta plantilla en el exploratorio anterior
# Lista de archivos en el directorio
archivos = os.listdir(directorio_defect)

# #imagenes decision
# pulgar_ok=cv2.imread(r'C:\Users\joano\Desktop\vision artificial juan olivan\mi codigo\proyecto_final\casting_cat\decision\pulgar_ok.png')
# pulgar_nok=cv2.imread(r'C:\Users\joano\Desktop\vision artificial juan olivan\mi codigo\proyecto_final\casting_cat\decision\pulgar_nok.png')
# Iteramos sobre cada archivo en el directorio
for archivo in archivos:
    # Comprobamos si el archivo es una imagen (extensión .jpg o .png)
    if archivo.endswith(".jpeg"):
        # Construimos la ruta completa de la imagen
        imagen_path = os.path.join(directorio_defect, archivo)
        
        # Cargamos la imagen
        imagen = cv2.imread(imagen_path)
        template=cv2.imread(template_path)
      
        # Convertimos la imagen a escala de grises
        imgGray = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
        template=cv2.cvtColor(template,cv2.COLOR_BGR2GRAY)
        
        img2=imagen.copy()
        w,h=template.shape[::-1]
        res=cv2.matchTemplate(imgGray,template,cv2.TM_CCOEFF_NORMED)
        umbralX=70/100 #este valor viene definido por el exploratorio anterior
        rectangulo=np.where(res>=umbralX)
        
        for puntos in zip(*rectangulo[::-1]):
 
                cv2.rectangle(img2,puntos,(puntos[0]+w,puntos[1]+h),(0,255,0),2)

            
        # Mostramos la imagen original y la imagen con los defectos detectados
        cv2.imshow("Imagen original", img2)
     
        
        # Esperamos una tecla para pasar a la siguiente imagen
        tecla = cv2.waitKey(0)
        
        # Si se presiona la tecla 'q' (quit), salimos del bucle
        if tecla == ord('q'):
            break

cv2.destroyAllWindows()


# Conclusión

Como hemos podido observar en los dos casos anteriores el que ha dado mejores resultado en la búsqueda del defecto ha sido el metodo de contornos, esto es debido sin dudar a que el patrón encontrado en la imagen de referencia cambia en cada imagen, esto provoca que la búsqueda no se haga correctamente y provoque falsos rechazos o no encontrar si quiera el defecto coincidente. Por ello, cabe decir que la mejor solución del sistema en el caso que el cliente no tenga suficiente recursos, se podría implementar el metodo de contornos para encontrar dicha solución, en este caso se deberia depurar más solución, aunque ya damos buenos resultados. En el caso que el cliente quisiera que la tasa de rechazo fuera mucho menor cabría decir que es una aplicación pensada para hacer con deep learning. En los proximos cursos daremos soluciones más exactas teniendo en cuenta el ML y el DL.